In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [3]:
cv=LeaveOneOut()

In [4]:
#X = np.loadtxt("1.txt")
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [5]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    param1=["best","random"]
    param_grid={"splitter":param1}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["splitter"]
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
    param_grid={"max_depth":param2}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["max_depth"]
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,50))  ## 设置max_features范围
    param_grid={"max_features":param3}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
#     print(datetime.datetime.now())
#     print("best parameters:{}".format(grid_search.best_params_))                        #输出最优网格化参数
#     print("best cross-validation score:{:.3f}".format(grid_search.best_score_))         #成员提供优化过程中观察到的最好评分
#     print("best estimator:\n{}".format(grid_search.best_estimator_))
    Predict_Y=[]    #存放预测值
    Prob_Y=[]       #存放预测概率
    Really_Y=y  #存放真实标签
    for train_index,test_index in cv.split(X):
        estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=grid_search.best_params_["max_features"])
        estimator.fit(X[train_index],y[train_index])
        Predict_Y.append(estimator.predict(X[test_index])[0])
        Prob_Y.append(estimator.predict_proba(X[test_index]))
    Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
    Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
    Prob_Y=pd.DataFrame(Prob_Y)
    Prob_Y.columns=["0","1"]
    confusion_matrix(Really_Y,Predict_Y)
    TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
    FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
    FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
    TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
    ACC=accuracy_score(Really_Y,Predict_Y)
    F1_score=f1_score(Really_Y,Predict_Y)
    AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
    Recall=recall_score(Really_Y,Predict_Y)
    Precision=precision_score(Really_Y,Predict_Y)
    S=p1
    D=p2
    M=grid_search.best_params_["max_features"]
#     print("ACC:",ACC)
#     print("f1_score:",F1_score)
#     print("AUC:",AUC)
    return TN,FN,FP,TP,ACC,Recall,Precision,F1_score,AUC,S,D,M

In [7]:
file_path = 'E:\datatest\PseDPC\d1\d11' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
111.csv
**************************************************
2022-02-13 23:49:37.309666
2022-02-13 23:49:38.428671
2022-02-13 23:49:43.259757
['111.csv' '67' '14' '21' '71' '0.7976878612716763' '0.8352941176470589'
 '0.7717391304347826' '0.8022598870056498' '0.8072860962566846' 'best' '5'
 '60']
##################################################
112.csv
**************************************************
2022-02-13 23:49:45.005076
2022-02-13 23:49:46.023352
2022-02-13 23:49:50.699840
['112.csv' '75' '14' '13' '71' '0.8439306358381503' '0.8352941176470589'
 '0.8452380952380952' '0.8402366863905326' '0.814505347593583' 'best' '4'
 '60']
##################################################
113.csv
**************************************************
2022-02-13 23:49:52.576851
2022-02-13 23:49:53.611050
2022-02-13 23:49:58.738354
['113.csv' '72' '16' '16' '69' '0.815028901734104' '0.8117647058823529'
 '0.8117647058823529' '0.8117647058823529' '0.

In [8]:
file_path = 'E:\datatest\PseDPC\d1\d12' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
121.csv
**************************************************
2022-02-13 23:50:37.580410
2022-02-13 23:50:38.645561
2022-02-13 23:50:40.729008
['121.csv' '68' '23' '20' '62' '0.7514450867052023' '0.7294117647058823'
 '0.7560975609756098' '0.7425149700598803' '0.7602941176470589' 'random'
 '3' '60']
##################################################
122.csv
**************************************************
2022-02-13 23:50:41.643540
2022-02-13 23:50:42.693757
2022-02-13 23:50:44.831012
['122.csv' '69' '10' '19' '75' '0.8323699421965318' '0.8823529411764706'
 '0.7978723404255319' '0.8379888268156425' '0.8947860962566845' 'random'
 '4' '110']
##################################################
123.csv
**************************************************
2022-02-13 23:50:45.949021
2022-02-13 23:50:46.925442
2022-02-13 23:50:48.969968
['123.csv' '63' '19' '25' '66' '0.7456647398843931' '0.7764705882352941'
 '0.7252747252747253' '0.75' '0.7715240

In [9]:
file_path = 'E:\datatest\PseDPC\d1\d13' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
131.csv
**************************************************
2022-02-13 23:51:36.755006
2022-02-13 23:51:37.773303
2022-02-13 23:51:42.775930
['131.csv' '72' '17' '16' '68' '0.8092485549132948' '0.8'
 '0.8095238095238095' '0.8047337278106509' '0.7889037433155082' 'best' '3'
 '60']
##################################################
132.csv
**************************************************
2022-02-13 23:51:44.330760
2022-02-13 23:51:45.377957
2022-02-13 23:51:50.708673
['132.csv' '67' '20' '21' '65' '0.7630057803468208' '0.7647058823529411'
 '0.7558139534883721' '0.7602339181286549' '0.696524064171123' 'best' '5'
 '10']
##################################################
133.csv
**************************************************
2022-02-13 23:51:52.429071
2022-02-13 23:51:53.518189
2022-02-13 23:51:55.629537
['133.csv' '49' '9' '39' '76' '0.7225433526011561' '0.8941176470588236'
 '0.6608695652173913' '0.7599999999999999' '0.732419786096256

In [10]:
file_path = 'E:\datatest\PseDPC\d1\d14' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
141.csv
**************************************************
2022-02-13 23:52:44.803989
2022-02-13 23:52:45.986777
2022-02-13 23:52:48.178912
['141.csv' '61' '13' '27' '72' '0.7687861271676301' '0.8470588235294118'
 '0.7272727272727273' '0.7826086956521741' '0.7967914438502673' 'random'
 '6' '110']
##################################################
142.csv
**************************************************
2022-02-13 23:52:49.248052
2022-02-13 23:52:50.335143
2022-02-13 23:52:55.323796
['142.csv' '80' '21' '8' '64' '0.8323699421965318' '0.7529411764705882'
 '0.8888888888888888' '0.8152866242038216' '0.7890374331550802' 'best' '3'
 '110']
##################################################
143.csv
**************************************************
2022-02-13 23:52:57.068129
2022-02-13 23:52:58.280906
2022-02-13 23:53:03.635591
['143.csv' '72' '17' '16' '68' '0.8092485549132948' '0.8'
 '0.8095238095238095' '0.8047337278106509' '0.7576871657

In [11]:
file_path = 'E:\datatest\PseDPC\d1\d15' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
151.csv
**************************************************
2022-02-13 23:53:46.052713
2022-02-13 23:53:47.662300
2022-02-13 23:53:50.609423
['151.csv' '72' '21' '16' '64' '0.7861271676300579' '0.7529411764705882'
 '0.8' '0.7757575757575759' '0.7822192513368984' 'random' '4' '60']
##################################################
152.csv
**************************************************
2022-02-13 23:53:51.549898
2022-02-13 23:53:53.102745
2022-02-13 23:53:59.961394
['152.csv' '76' '23' '12' '62' '0.7976878612716763' '0.7294117647058823'
 '0.8378378378378378' '0.779874213836478' '0.7247326203208557' 'best' '3'
 '60']
##################################################
153.csv
**************************************************
2022-02-13 23:54:01.903198
2022-02-13 23:54:03.390266
2022-02-13 23:54:09.871199
['153.csv' '76' '22' '12' '63' '0.8034682080924855' '0.7411764705882353'
 '0.84' '0.7875000000000001' '0.7352941176470589' 'best' '3

In [12]:
file_path = 'E:\datatest\PseDPC\d1\d16' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
161.csv
**************************************************
2022-02-13 23:55:04.403299
2022-02-13 23:55:05.636000
2022-02-13 23:55:11.126313
['161.csv' '78' '25' '10' '60' '0.7976878612716763' '0.7058823529411765'
 '0.8571428571428571' '0.7741935483870968' '0.7844919786096257' 'best' '4'
 '60']
##################################################
162.csv
**************************************************
2022-02-13 23:55:12.844747
2022-02-13 23:55:14.052482
2022-02-13 23:55:19.462009
['162.csv' '73' '8' '15' '77' '0.8670520231213873' '0.9058823529411765'
 '0.8369565217391305' '0.8700564971751413' '0.8575534759358289' 'best' '3'
 '60']
##################################################
163.csv
**************************************************
2022-02-13 23:55:20.980968
2022-02-13 23:55:22.166773
2022-02-13 23:55:27.566326
['163.csv' '73' '8' '15' '77' '0.8670520231213873' '0.9058823529411765'
 '0.8369565217391305' '0.8700564971751413' '0.

In [13]:
file_path = 'E:\datatest\PseDPC\d1\d17' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
171.csv
**************************************************
2022-02-13 23:56:14.554609
2022-02-13 23:56:15.804266
2022-02-13 23:56:17.954513
['171.csv' '70' '13' '18' '72' '0.8208092485549133' '0.8470588235294118'
 '0.8' '0.822857142857143' '0.8255347593582887' 'random' '5' '60']
##################################################
172.csv
**************************************************
2022-02-13 23:56:18.895029
2022-02-13 23:56:20.136696
2022-02-13 23:56:22.317875
['172.csv' '68' '12' '20' '73' '0.815028901734104' '0.8588235294117647'
 '0.7849462365591398' '0.8202247191011236' '0.8311497326203208' 'random'
 '6' '110']
##################################################
173.csv
**************************************************
2022-02-13 23:56:23.334141
2022-02-13 23:56:24.442176
2022-02-13 23:56:29.650221
['173.csv' '77' '8' '11' '77' '0.8901734104046243' '0.9058823529411765'
 '0.875' '0.8901734104046243' '0.868716577540107' 'best' '

In [14]:
file_path = 'E:\datatest\PseDPC\d1\d18' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
181.csv
**************************************************
2022-02-13 23:57:17.660770
2022-02-13 23:57:18.771821
2022-02-13 23:57:23.833289
['181.csv' '68' '26' '20' '59' '0.7341040462427746' '0.6941176470588235'
 '0.7468354430379747' '0.7195121951219512' '0.6667112299465241' 'best' '5'
 '110']
##################################################
182.csv
**************************************************
2022-02-13 23:57:25.892774
2022-02-13 23:57:26.999804
2022-02-13 23:57:29.156014
['182.csv' '63' '20' '25' '65' '0.7398843930635838' '0.7647058823529411'
 '0.7222222222222222' '0.7428571428571428' '0.7533422459893048' 'random'
 '8' '110']
##################################################
183.csv
**************************************************
2022-02-13 23:57:30.217174
2022-02-13 23:57:31.322218
2022-02-13 23:57:33.439553
['183.csv' '52' '9' '36' '76' '0.7398843930635838' '0.8941176470588236'
 '0.6785714285714286' '0.7715736040609137

In [16]:
file_path = 'E:\datatest\PseDPC\d1\d19' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
191.csv
**************************************************
2022-02-14 00:05:51.759646
2022-02-14 00:05:53.096061
2022-02-14 00:05:59.211698
['191.csv' '68' '9' '20' '76' '0.8323699421965318' '0.8941176470588236'
 '0.7916666666666666' '0.839779005524862' '0.8387032085561498' 'best' '3'
 '110']
##################################################
192.csv
**************************************************
2022-02-14 00:06:00.938079
2022-02-14 00:06:02.099971
2022-02-14 00:06:04.324020
['192.csv' '59' '24' '29' '61' '0.6936416184971098' '0.7176470588235294'
 '0.6777777777777778' '0.6971428571428572' '0.7191176470588235' 'random'
 '7' '110']
##################################################
193.csv
**************************************************
2022-02-14 00:06:05.485912
2022-02-14 00:06:06.768480
2022-02-14 00:06:09.327662
['193.csv' '71' '24' '17' '61' '0.7630057803468208' '0.7176470588235294'
 '0.782051282051282' '0.7484662576687117' 

In [17]:
file_path = 'E:\datatest\PseDPC\d1\d110' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1101.csv
**************************************************
2022-02-14 00:06:45.863881
2022-02-14 00:06:47.787733
2022-02-14 00:06:51.034082
['1101.csv' '64' '12' '24' '73' '0.791907514450867' '0.8588235294117647'
 '0.7525773195876289' '0.8021978021978021' '0.6661096256684492' 'random'
 '3' '60']
##################################################
1102.csv
**************************************************
2022-02-14 00:06:52.148070
2022-02-14 00:06:53.443635
2022-02-14 00:06:59.487463
['1102.csv' '69' '13' '19' '72' '0.815028901734104' '0.8470588235294118'
 '0.7912087912087912' '0.8181818181818181' '0.7998663101604279' 'best' '3'
 '60']
##################################################
1103.csv
**************************************************
2022-02-14 00:07:01.179903
2022-02-14 00:07:02.560240
2022-02-14 00:07:08.736687
['1103.csv' '68' '18' '20' '67' '0.7803468208092486' '0.788235294117647'
 '0.7701149425287356' '0.77906976744186

In [18]:
file_path = 'E:\datatest\PseDPC\d1\d111' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1111.csv
**************************************************
2022-02-14 00:08:07.556313
2022-02-14 00:08:09.011447
2022-02-14 00:08:15.016376
['1111.csv' '71' '13' '17' '72' '0.8265895953757225' '0.8470588235294118'
 '0.8089887640449438' '0.8275862068965517' '0.8115641711229946' 'best' '3'
 '110']
##################################################
1112.csv
**************************************************
2022-02-14 00:08:17.025007
2022-02-14 00:08:18.304556
2022-02-14 00:08:24.867026
['1112.csv' '69' '14' '19' '71' '0.8092485549132948' '0.8352941176470589'
 '0.7888888888888889' '0.8114285714285714' '0.791644385026738' 'best' '3'
 '110']
##################################################
1113.csv
**************************************************
2022-02-14 00:08:26.598400
2022-02-14 00:08:27.804170
2022-02-14 00:08:33.874899
['1113.csv' '68' '15' '20' '70' '0.7976878612716763' '0.8235294117647058'
 '0.7777777777777778' '0.799999999999

In [19]:
file_path = 'E:\datatest\PseDPC\d1\d112' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1121.csv
**************************************************
2022-02-14 00:09:29.462176
2022-02-14 00:09:31.293277
2022-02-14 00:09:38.497002
['1121.csv' '67' '17' '21' '68' '0.7803468208092486' '0.8'
 '0.7640449438202247' '0.7816091954022989' '0.7635026737967914' 'best' '5'
 '10']
##################################################
1122.csv
**************************************************
2022-02-14 00:09:40.251308
2022-02-14 00:09:41.450135
2022-02-14 00:09:47.490971
['1122.csv' '69' '16' '19' '69' '0.7976878612716763' '0.8117647058823529'
 '0.7840909090909091' '0.7976878612716763' '0.8025401069518716' 'best' '5'
 '60']
##################################################
1123.csv
**************************************************
2022-02-14 00:09:49.428754
2022-02-14 00:09:50.669434
2022-02-14 00:09:56.311367
['1123.csv' '71' '17' '17' '68' '0.8034682080924855' '0.8' '0.8'
 '0.8000000000000002' '0.8282085561497328' 'best' '4' '60']
##

In [20]:
file_path = 'E:\datatest\PseDPC\d1\d113' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1131.csv
**************************************************
2022-02-14 00:10:26.767887
2022-02-14 00:10:28.009533
2022-02-14 00:10:33.813004
['1131.csv' '68' '14' '20' '71' '0.8034682080924855' '0.8352941176470589'
 '0.7802197802197802' '0.8068181818181818' '0.7713903743315508' 'best' '4'
 '110']
##################################################
1132.csv
**************************************************
2022-02-14 00:10:35.794703
2022-02-14 00:10:37.111180
2022-02-14 00:10:43.185927
['1132.csv' '74' '17' '14' '68' '0.8208092485549133' '0.8'
 '0.8292682926829268' '0.8143712574850299' '0.7685828877005347' 'best' '3'
 '110']
##################################################
1133.csv
**************************************************
2022-02-14 00:10:44.936281
2022-02-14 00:10:46.246766
2022-02-14 00:10:52.282617
['1133.csv' '77' '16' '11' '69' '0.8439306358381503' '0.8117647058823529'
 '0.8625' '0.8363636363636364' '0.803409090909091' 

In [21]:
file_path = 'E:\datatest\PseDPC\d1\d114' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1141.csv
**************************************************
2022-02-14 00:11:50.184670
2022-02-14 00:11:51.738511
2022-02-14 00:11:58.305968
['1141.csv' '70' '9' '18' '76' '0.8439306358381503' '0.8941176470588236'
 '0.8085106382978723' '0.8491620111731844' '0.8417112299465241' 'best' '3'
 '110']
##################################################
1142.csv
**************************************************
2022-02-14 00:12:00.227798
2022-02-14 00:12:01.588188
2022-02-14 00:12:08.289230
['1142.csv' '74' '9' '14' '76' '0.8670520231213873' '0.8941176470588236'
 '0.8444444444444444' '0.8685714285714287' '0.8504679144385027' 'best' '3'
 '60']
##################################################
1143.csv
**************************************************
2022-02-14 00:12:10.034588
2022-02-14 00:12:11.626302
2022-02-14 00:12:18.475975
['1143.csv' '80' '12' '8' '73' '0.884393063583815' '0.8588235294117647'
 '0.9012345679012346' '0.8795180722891566

In [22]:
file_path = 'E:\datatest\PseDPC\d1\d115' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1151.csv
**************************************************
2022-02-14 00:13:20.101099
2022-02-14 00:13:21.503347
2022-02-14 00:13:28.227357
['1151.csv' '76' '17' '12' '68' '0.8323699421965318' '0.8' '0.85'
 '0.8242424242424243' '0.7768716577540107' 'best' '3' '60']
##################################################
1152.csv
**************************************************
2022-02-14 00:13:30.074414
2022-02-14 00:13:31.412833
2022-02-14 00:13:37.642167
['1152.csv' '69' '17' '19' '68' '0.791907514450867' '0.8'
 '0.7816091954022989' '0.7906976744186047' '0.7636363636363637' 'best' '4'
 '110']
##################################################
1153.csv
**************************************************
2022-02-14 00:13:39.987892
2022-02-14 00:13:41.402107
2022-02-14 00:13:48.122371
['1153.csv' '71' '18' '17' '67' '0.7976878612716763' '0.788235294117647'
 '0.7976190476190477' '0.7928994082840236' '0.7896390374331551' 'best' '4'
 '110']
#

In [23]:
file_path = 'E:\datatest\PseDPC\d1\d116' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1161.csv
**************************************************
2022-02-14 00:14:41.151977
2022-02-14 00:14:42.935206
2022-02-14 00:14:49.414897
['1161.csv' '76' '12' '12' '73' '0.861271676300578' '0.8588235294117647'
 '0.8588235294117647' '0.8588235294117647' '0.8120320855614974' 'best' '3'
 '60']
##################################################
1162.csv
**************************************************
2022-02-14 00:14:51.031544
2022-02-14 00:14:52.207398
2022-02-14 00:14:59.144839
['1162.csv' '74' '14' '14' '71' '0.838150289017341' '0.8352941176470589'
 '0.8352941176470589' '0.8352941176470589' '0.7538770053475936' 'best' '3'
 '60']
##################################################
1163.csv
**************************************************
2022-02-14 00:15:01.457681
2022-02-14 00:15:03.123193
2022-02-14 00:15:10.079581
['1163.csv' '76' '14' '12' '71' '0.8497109826589595' '0.8352941176470589'
 '0.8554216867469879' '0.845238095238095

In [24]:
file_path = 'E:\datatest\PseDPC\d1\d117' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1171.csv
**************************************************
2022-02-14 00:16:08.306796
2022-02-14 00:16:09.646244
2022-02-14 00:16:15.780798
['1171.csv' '68' '12' '20' '73' '0.815028901734104' '0.8588235294117647'
 '0.7849462365591398' '0.8202247191011236' '0.7832887700534761' 'best' '3'
 '60']
##################################################
1172.csv
**************************************************
2022-02-14 00:16:17.387499
2022-02-14 00:16:18.657102
2022-02-14 00:16:24.571278
['1172.csv' '73' '9' '15' '76' '0.861271676300578' '0.8941176470588236'
 '0.8351648351648352' '0.8636363636363636' '0.8816844919786097' 'best' '3'
 '110']
##################################################
1173.csv
**************************************************
2022-02-14 00:16:26.347549
2022-02-14 00:16:27.682954
2022-02-14 00:16:33.776671
['1173.csv' '68' '10' '20' '75' '0.8265895953757225' '0.8823529411764706'
 '0.7894736842105263' '0.833333333333333

In [25]:
file_path = 'E:\datatest\PseDPC\d1\d118' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1181.csv
**************************************************
2022-02-14 00:17:31.543097
2022-02-14 00:17:32.786769
2022-02-14 00:17:38.504471
['1181.csv' '78' '15' '10' '70' '0.8554913294797688' '0.8235294117647058'
 '0.875' '0.8484848484848485' '0.8736631016042781' 'best' '6' '110']
##################################################
1182.csv
**************************************************
2022-02-14 00:17:40.851193
2022-02-14 00:17:42.111820
2022-02-14 00:17:48.502743
['1182.csv' '72' '9' '16' '76' '0.8554913294797688' '0.8941176470588236'
 '0.8260869565217391' '0.8587570621468926' '0.843716577540107' 'best' '3'
 '110']
##################################################
1183.csv
**************************************************
2022-02-14 00:17:50.381715
2022-02-14 00:17:51.639329
2022-02-14 00:17:57.503672
['1183.csv' '72' '10' '16' '75' '0.8497109826589595' '0.8823529411764706'
 '0.8241758241758241' '0.8522727272727273' '0.780280

In [26]:
file_path = 'E:\datatest\PseDPC\d1\d119' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
1191.csv
**************************************************
2022-02-14 00:18:55.049675
2022-02-14 00:18:56.297336
2022-02-14 00:18:58.527370
['1191.csv' '60' '10' '28' '75' '0.7803468208092486' '0.8823529411764706'
 '0.7281553398058253' '0.7978723404255318' '0.7364973262032086' 'random'
 '3' '110']
##################################################
1192.csv
**************************************************
2022-02-14 00:18:59.407044
2022-02-14 00:19:00.718508
2022-02-14 00:19:06.804247
['1192.csv' '67' '11' '21' '74' '0.815028901734104' '0.8705882352941177'
 '0.7789473684210526' '0.8222222222222222' '0.7293449197860962' 'best' '3'
 '60']
##################################################
1193.csv
**************************************************
2022-02-14 00:19:08.388985
2022-02-14 00:19:09.760347
2022-02-14 00:19:16.004611
['1193.csv' '71' '14' '17' '71' '0.8208092485549133' '0.8352941176470589'
 '0.8068181818181818' '0.82080924855

In [ ]:
file_path = 'E:\datatest\PseDPC\d1\d120' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')